# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [15]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [16]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [19]:
# print names of all countries
for child in document_tree.getroot():
    print(child.find('name').text)

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [25]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print('* ' + element.find('name').text + ':')
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print(capitals_string[:-2])

* Albania:
Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece:
Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia:
Skopje, Kumanovo
* Serbia:
Beograd, Novi Sad, Niš
* Montenegro:
Podgorica
* Kosovo:
Prishtine
* Andorra:
Andorra la Vella


## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [26]:
document = ET.parse( './data/mondial_database.xml' )

# 1. 10 countries with the lowest infant mortality rates

In [216]:
import pandas as pd
import numpy as np

# Extract countries, population, and infant_mortality
country_dict = {}
for element in document.iterfind('country'): #find all countries
    for population in element.iterfind('population'): #find all population
        try:
             country_dict[element.find('name').text] = [int(population.text), float(element.find('infant_mortality').text)] #find infant_mortality stats
        except: 
            pass
        
#print(country_dict)

In [217]:
# Convert dictionary into a dataframe
country = pd.DataFrame.from_dict(country_dict, orient='index') 

# Reset index
country.reset_index(drop=False, inplace=True)

# Update column names
country.columns = ['Country','Population', 'Infant_Mortality']

# Subset data
country = country[['Country','Infant_Mortality']]
print(country.head(10))

          Country  Infant_Mortality
0         Albania             13.19
1          Greece              4.78
2       Macedonia              7.90
3          Serbia              6.16
4         Andorra              3.69
5          France              3.31
6           Spain              3.33
7         Austria              4.16
8  Czech Republic              2.63
9         Germany              3.46


In [94]:
# Sort 10 countries with the lowest Infant_Mortality rate
country.sort_values(by='Infant_Mortality', ascending=True).head(10)

,Country,Infant_Mortality
36,Monaco,1.81
90,Japan,2.13
109,Bermuda,2.48
34,Norway,2.48
98,Singapore,2.53
35,Sweden,2.60
8,Czech Republic,2.63
72,Hong Kong,2.73
73,Macao,3.13
39,Iceland,3.15


# 2. 10 cities with the largest population

In [212]:
# Extract cities and populations
city_dict = {}
for element in document.iterfind('country'):
    for city in element.iterfind('city'):
        try:
             city_dict[city.find('name').text] = int(city.find('population').text)
        except: 
            pass

# Convert dictionary into a dataframe
cities_LgPop = pd.DataFrame.from_dict(city_dict, orient='index') 

# Reset index
cities_LgPop.reset_index(drop=False, inplace=True)

# Update column names
cities_LgPop.columns = ['City','Population']

# Sort 10 cities with the largest population
cities_LgPop.sort_values(by='Population', ascending=False).head(10)

,City,Population
165,Seoul,10229262
123,Hong Kong,7055071
154,Al Qahirah,6053000
75,Bangkok,5876000
87,Ho Chi Minh,3924435
166,Busan,3813814
205,New Taipei,3722082
84,Hanoi,3056146
153,Al Iskandariyah,2917000
204,Taipei,2626138


# 3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)

In [633]:
# Create country, ethnic group, and percentage dataframe
ethnic_groups = [] #create a list called ethnicgroups
for element in document.iterfind('country'): #find all countries
    country = element.find('name').text #find the name
    for subelement in element.iterfind('ethnicgroup'): #find all ethinicgroup
        group = []  #create a list called group
        group.append(country) #add country to the group list
        
        if  pd.isnull(subelement):    #if subelement ('ethnicgroup') is null, move to next record
            continue
        else:
            group.append(subelement.text) #add subelement ('ethnicgroup') to group list
            if pd.isnull(subelement.attrib['percentage']):  #if subelement attrib['percentage'] is null, move to next record
                group.append(replace)
            else:
                group.append(subelement.attrib['percentage']) #else add the subelement attrib['percentage'] to the list

        ethnic_groups.append(group) #add the group list to the enthinicgroups list

# Convert list into a dataframe
ethnicgroups = pd.DataFrame(ethnic_groups) 

# Update column names
ethnicgroups.columns =['Country','EthnicGroup','Pop_Pct'] 

In [634]:
# Create population dataframe
import pandas as pd
import numpy as np

# Extract country and population into a dictionary
pop_dict = {}
for element in document.iterfind('country'): #find all countries
    for population in element.iterfind('population'): #find all population
        try:
             pop_dict[element.find('name').text] = [int(population.text)]
        except: 
            pass
        
# Convert dictionary into a dataframe
population = pd.DataFrame.from_dict(pop_dict, orient='index') 

# Reset index
population.reset_index(drop=False, inplace=True)

# Update column names
population.columns = ['Country','Population']

#print(population.head(10))

In [715]:
# Merge ethnic_groups & Population dataframes       
final_ethnicgroups = ethnicgroups.merge(population, how='left', on='Country')

# Create a new field 'EthnicGroup_Pct' and calculate percentage of Ethinic Groups
final_ethnicgroups['EthnicGroup_Pct'] = (final_ethnicgroups['Pop_Pct'].astype(float)/100) * final_ethnicgroups['Population']

#final_ethnicgroups.head(10)

# Group ethnic_group
final_ethnicgroups = final_ethnicgroups.groupby('EthnicGroup').sum()

# Reset index
final_ethnicgroups.reset_index(drop=False, inplace=True)

# Sort by descending EthnicGroup_Pct
final_ethnicgroups.sort_values(by='EthnicGroup_Pct', ascending=False, inplace=True)

# Top 10 ethnic groups with largest percentage of overall populations
final_ethnicgroups.head(10)

,EthnicGroup,Population,EthnicGroup_Pct
113,Han Chinese,1360720000,1.245059e+09
120,Indo-Aryan,1210854977,8.718156e+08
89,European,1157295639,4.948722e+08
2,African,975352746,3.183251e+08
77,Dravidian,1210854977,3.027137e+08
176,Mestizo,279743964,1.577344e+08
42,Bengali,149772364,1.467769e+08
217,Russian,322438406,1.318570e+08
128,Japanese,127298000,1.265342e+08
163,Malay,377500275,1.219936e+08


# 4a. name and country of longest river  

In [732]:
river_list = []
for element in document.iterfind('river'): # find all river
    rivername = element.find('name').text #find the name
    for subelement in element.iterfind('located'): #find located country
        try:
            group = [subelement.attrib['country'], element.find('name').text, int(element.find('length').text)] # extract country, river, and river length
            river_list.append(group) #add to river list
        except:
            pass

# Create a dataframe containing country code, river, and river length    
columns = ['Country_Cd', 'River', 'River_Length']
rivers = pd.DataFrame(columns=columns)
rivers = rivers.append(pd.DataFrame(river_list, columns=columns), ignore_index=True)

# Sort by descending River_Length
rivers.sort_values(by='River_Length', ascending=False, inplace=True)

# Review top 10 records
rivers.head(10)

,Country_Cd,River,River_Length
243,CO,Amazonas,6448.0
244,BR,Amazonas,6448.0
245,PE,Amazonas,6448.0
191,CN,Jangtse,6380.0
190,CN,Hwangho,4845.0
170,R,Lena,4400.0
282,RCB,Zaire,4374.0
281,ZRE,Zaire,4374.0
192,CN,Mekong,4350.0
160,CN,Irtysch,4248.0


In [733]:
# Create country code & country dictionary
country_dict = {}
for element in document.iterfind('country'):
    country_dict[element.attrib['car_code']] = element.find('name').text

# Convert dictionary into a dataframe
country_cd = pd.DataFrame.from_dict(country_dict, orient='index') 

# Reset index
country_cd.reset_index(drop=False, inplace=True)

# Update column names
country_cd.columns = ['Country_Cd','Country']

In [734]:
# Merge rivers & country dataframes       
finalrivers = rivers.merge(country_cd, how='left', on='Country_Cd')

# Review top 5 records
finalrivers.head()

,Country_Cd,River,River_Length,Country
0,CO,Amazonas,6448.0,Colombia
1,BR,Amazonas,6448.0,Brazil
2,PE,Amazonas,6448.0,Peru
3,CN,Jangtse,6380.0,China
4,CN,Hwangho,4845.0,China


In [735]:
# Re-arrange columns
longest_rivers = finalrivers[['Country_Cd','Country','River','River_Length']]

# Select longest river
longest_rivers = longest_rivers[0:3]

# Display results
longest_rivers

,Country_Cd,Country,River,River_Length
0,CO,Colombia,Amazonas,6448.0
1,BR,Brazil,Amazonas,6448.0
2,PE,Peru,Amazonas,6448.0


# 4b. name and country of largest lake   

In [745]:
lake_list = []
for element in document.iterfind('lake'): # find all lakes
    lakename = element.find('name').text #find the name
    for subelement in element.iterfind('located'): #find located country
        try:
            group = [subelement.attrib['country'], element.find('name').text, int(element.find('area').text)] # extract country, lakes, and lakes area
            lake_list.append(group) #add to lakes list
        except:
            pass

# Create a dataframe containing country code, lake, and lake area    
columns = ['Country_Cd', 'Lake', 'Lake_Size']
lakes = pd.DataFrame(columns=columns)
lakes = lakes.append(pd.DataFrame(lake_list, columns=columns), ignore_index=True)

# Sort by descending River_Length
lakes.sort_values(by='Lake_Size', ascending=False, inplace=True)

# Review top 10 records
lakes.head(10)

,Country_Cd,Lake,Lake_Size
47,R,Caspian Sea,386400.0
48,KAZ,Caspian Sea,386400.0
49,IR,Caspian Sea,386400.0
50,TM,Caspian Sea,386400.0
107,USA,Lake Superior,82103.0
106,CDN,Lake Superior,82103.0
73,EAT,Lake Victoria,68870.0
102,CDN,Lake Huron,59600.0
103,USA,Lake Huron,59600.0
105,USA,Lake Michigan,57800.0


In [746]:
# Create country code & country dictionary
country_dict = {}
for element in document.iterfind('country'):
    country_dict[element.attrib['car_code']] = element.find('name').text

# Convert dictionary into a dataframe
country_cd = pd.DataFrame.from_dict(country_dict, orient='index') 

# Reset index
country_cd.reset_index(drop=False, inplace=True)

# Update column names
country_cd.columns = ['Country_Cd','Country']

In [747]:
# Merge lake & country dataframes       
finallakes = lakes.merge(country_cd, how='left', on='Country_Cd')

# Review top 5 records
finallakes.head()

,Country_Cd,Lake,Lake_Size,Country
0,R,Caspian Sea,386400.0,Russia
1,KAZ,Caspian Sea,386400.0,Kazakhstan
2,IR,Caspian Sea,386400.0,Iran
3,TM,Caspian Sea,386400.0,Turkmenistan
4,USA,Lake Superior,82103.0,United States


In [748]:
# Re-arrange columns
largest_lakes = finallakes[['Country_Cd','Country','Lake','Lake_Size']]

# Select largest lake
largest_lakes = largest_lakes[0:4]

# Display results
largest_lakes

,Country_Cd,Country,Lake,Lake_Size
0,R,Russia,Caspian Sea,386400.0
1,KAZ,Kazakhstan,Caspian Sea,386400.0
2,IR,Iran,Caspian Sea,386400.0
3,TM,Turkmenistan,Caspian Sea,386400.0


# 4c. name and country of airport at highest elevation

In [762]:
airport_list = []
for element in document.iterfind('airport'): # find all airport
    airportname = element.find('name').text #find the name
    try:
        group = [subelement.attrib['country'], element.find('name').text, int(element.find('elevation').text)] # extract country & elevation
        airport_list.append(group) #add to airport list
    except:
        pass

# Create a dataframe containing country code, airport, and elevation    
columns = ['Country_Cd', 'Airport', 'Elevation']
airports = pd.DataFrame(columns=columns)
airports = airports.append(pd.DataFrame(airport_list, columns=columns), ignore_index=True)

# Sort by descending Elevation
airports.sort_values(by='Elevation', ascending=False, inplace=True)

# Review top 10 records
airports.head(10)

,Country_Cd,Airport,Elevation
80,AUS,El Alto Intl,4063.0
212,AUS,Lhasa-Gonggar,4005.0
230,AUS,Yushu Batang,3963.0
787,AUS,Juliaca,3827.0
789,AUS,Teniente Alejandro Velasco Astete Intl,3311.0
82,AUS,Juana Azurduy De Padilla,2905.0
308,AUS,Mariscal Sucre Intl,2813.0
779,AUS,Coronel Fap Alfredo Mendivil Duarte,2719.0
781,AUS,Mayor General FAP Armando Revoredo Iglesias Ai...,2677.0
666,AUS,Licenciado Adolfo Lopez Mateos Intl,2581.0


In [763]:
# Create country code & country dictionary
country_dict = {}
for element in document.iterfind('country'):
    country_dict[element.attrib['car_code']] = element.find('name').text

# Convert dictionary into a dataframe
country_cd = pd.DataFrame.from_dict(country_dict, orient='index') 

# Reset index
country_cd.reset_index(drop=False, inplace=True)

# Update column names
country_cd.columns = ['Country_Cd','Country']

In [768]:
# Merge airports & country dataframes       
finalairports = airports.merge(country_cd, how='left', on='Country_Cd')

# Review top 5 records
finalairports.head()

,Country_Cd,Airport,Elevation,Country
0,AUS,El Alto Intl,4063.0,Australia
1,AUS,Lhasa-Gonggar,4005.0,Australia
2,AUS,Yushu Batang,3963.0,Australia
3,AUS,Juliaca,3827.0,Australia
4,AUS,Teniente Alejandro Velasco Astete Intl,3311.0,Australia


In [770]:
# Re-arrange columns
highestelevation = finalairports[['Country_Cd','Country','Airport','Elevation']]

# Select airport with highest elevation
highestelevation = highestelevation[0:1]

# Display results
highestelevation

,Country_Cd,Country,Airport,Elevation
0,AUS,Australia,El Alto Intl,4063.0
